In [3]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from nolearn.dbn import DBN
from skimage import io
from skimage import data, segmentation, filters, color, img_as_float, img_as_ubyte, exposure, feature, measure, morphology
from skimage.color import rgb2gray
from skimage.morphology import square
from skimage.feature import hog
import cv2
import numpy as np
import cv2
from time import time
from glob import glob
import os

gnumpy: failed to import cudamat. Using npmat instead. No GPU will be used.


In [ ]:
### Plan2 ###

In [217]:
features = joblib.load("./mldata/features_1200.mat")
labels = joblib.load("./mldata/lables_1200.mat")

features = np.array(features, 'int16')
labels = np.array(labels, 'int')

In [218]:
t0 = time()
def scale(X, eps = 0.001):
    # scale the data points s.t the columns of the feature space
    # (i.e the predictors) are within the range [0, 1]
    return (X - np.min(X, axis = 0)) / (np.max(X, axis = 0) + eps)

features = features.astype("float32")
features = scale(features)

print "escape time : ", round(time()-t0, 3), "s"

escape time :  0.091 s


In [219]:
# scale the data to the range [0, 1] and then construct the training
# and testing splits
(trainX, testX, trainY, testY) = train_test_split(features, labels, test_size = 0.1)
print "the shape of training set %s rows, %s columns" %(trainX.shape[0], trainX.shape[1])
print "the shape of test set %s rows, %s columns" %(testX.shape[0], testX.shape[1])
print "the range of training set : %s ~ %s" %(trainX.min(),trainX.max())
print "the range of test set : %s ~ %s" %(testX.min(),testX.max())

the shape of training set 5400 rows, 784 columns
the shape of test set 600 rows, 784 columns
the range of training set : 0.0 ~ 0.999996
the range of test set : 0.0 ~ 0.999996


In [220]:
# train the Deep Belief Network with 784 input units (the flattened,
# 28x28 grayscale image), 500 hidden units, 9 output units (one for
# each possible output classification, which are the digits 0-8)
t0 = time()
dbn = DBN(
    [trainX.shape[1], 500, 9],
    learn_rates = 0.009,
    learn_rate_decays = 0.9,
    epochs = 10,
    verbose = 1)
dbn.fit(trainX, trainY)
print "escape time : ", round(time()-t0, 3), "s"

[DBN] fitting X.shape=(5400, 784)
[DBN] layers [784, 500, 9]
[DBN] Fine-tune...

100%



Epoch 1:

100%



  loss 0.463643961276
  err  0.115513392857
  (0:00:01)
Epoch 2:

...


  loss 0.214639534109
  err  0.0574776785714
  (0:00:01)
Epoch 8:

100%



  loss 0.235282459518
  err  0.0638020833333
  (0:00:01)
Epoch 9:

100%



  loss 0.20613166548
  err  0.0580357142857
  (0:00:01)
Epoch 10:
  loss 0.208449641597
  err  0.0556175595238
  (0:00:01)
escape time :  12.585 s


In [221]:
# compute the predictions for the test data and show a classification report
preds = dbn.predict(testX)
print "accuracy score : %s" %(accuracy_score(testY, preds))
print "classification report : "
print classification_report(testY, preds)
print "confusion matrix : "
print confusion_matrix(testY, preds)

accuracy score : 0.95
classification report : 
             precision    recall  f1-score   support

          0       0.96      0.93      0.94        70
          1       0.93      0.96      0.94        93
          2       0.86      0.93      0.89        55
          3       0.98      0.95      0.96        58
          4       0.98      0.98      0.98        64
          5       0.98      0.94      0.96        62
          6       0.90      0.84      0.87        45
          7       1.00      0.99      0.99       100
          8       0.91      0.98      0.95        53

avg / total       0.95      0.95      0.95       600

confusion matrix : 
[[65  0  3  0  0  0  1  0  1]
 [ 2 89  1  0  0  0  0  0  1]
 [ 0  1 51  1  0  0  1  0  1]
 [ 0  1  2 55  0  0  0  0  0]
 [ 0  0  0  0 63  0  0  0  1]
 [ 0  2  1  0  0 58  1  0  0]
 [ 1  3  1  0  0  1 38  0  1]
 [ 0  0  0  0  0  0  1 99  0]
 [ 0  0  0  0  1  0  0  0 52]]


In [1]:
# MNIST TEST

In [4]:
dataset = datasets.fetch_mldata("MNIST Original")

# scale the data to the range [0, 1] and then construct the training
# and testing splits
(trainX, testX, trainY, testY) = train_test_split(dataset.data / 255.0, dataset.target.astype("int0"), test_size = 0.33)

In [16]:
# train the Deep Belief Network with 784 input units (the flattened,
# 28x28 grayscale image), 300 hidden units, 10 output units (one for
# each possible output classification, which are the digits 1-10)
dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.01,
    learn_rate_decays = 0.9,
    epochs = 1000,
    verbose = 1)
dbn.fit(trainX, trainY)

# compute the predictions for the test data and show a classification report
preds = dbn.predict(testX)
print "accuracy score : %s" %(accuracy_score(testY, preds))
print "classification report : "
print classification_report(testY, preds)
print "confusion matrix : "
print confusion_matrix(testY, preds)

[DBN] fitting X.shape=(46900, 784)
[DBN] layers [784, 300, 10]
[DBN] Fine-tune...

100%



Epoch 1:

100%



  loss 0.435460707254
  err  0.121285860656
  (0:00:07)
Epoch 2:

...


  loss 0.0519130192175
  err  0.0117827868852
  (0:00:10)
Epoch 998:

100%



  loss 0.0503042559248
  err  0.0118468237705
  (0:00:10)
Epoch 999:

100%



  loss 0.0506622662114
  err  0.0120175887978
  (0:00:10)
Epoch 1000:
  loss 0.0506539636977
  err  0.0117827868852
  (0:00:10)
accuracy score : 0.97354978355
classification report : 
             precision    recall  f1-score   support

          0       0.98      0.98      0.98      2278
          1       0.98      0.99      0.98      2574
          2       0.97      0.97      0.97      2334
          3       0.97      0.96      0.97      2387
          4       0.97      0.97      0.97      2290
          5       0.97      0.97      0.97      2036
          6       0.98      0.99      0.98      2266
          7       0.98      0.96      0.97      2403
          8       0.97      0.96      0.97      2232
          9       0.96      0.97      0.96      2300

avg / total       0.97      0.97      0.97     23100

confusion matrix : 
[[2241    1    5    2    1    6    8    5    6    3]
 [   0 2539   11    8    2    0    5    6    3    0]
 [  11    7 2265    8    7    3    8   14    6    

dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.3,
    learn_rate_decays = 0.9,
    epochs = 10,
    verbose = 1)
accuracy score : 0.977142857143

dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.3,
    learn_rate_decays = 0.9,
    epochs = 20,
    verbose = 1)
accuracy score : 0.980779220779

dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.3,
    learn_rate_decays = 0.9,
    epochs = 30,
    verbose = 1)
accuracy score : 0.981082251082

dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.4,
    learn_rate_decays = 0.9,
    epochs = 20,
    verbose = 1)
accuracy score : 0.979047619048

dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.2,
    learn_rate_decays = 0.9,
    epochs = 20,
    verbose = 1)
accuracy score : 0.981038961039

dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.5,
    learn_rate_decays = 0.9,
    epochs = 20,
    verbose = 1)
accuracy score : 0.971601731602

dbn = DBN(
    [trainX.shape[1], 400, 10],
    learn_rates = 0.3,
    learn_rate_decays = 0.9,
    epochs = 20,
    verbose = 1)
accuracy score : 0.981948051948

dbn = DBN(
    [trainX.shape[1], 500, 10],
    learn_rates = 0.3,
    learn_rate_decays = 0.9,
    epochs = 20,
    verbose = 1)
accuracy score : 0.981255411255

dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.001,
    learn_rate_decays = 0.9,
    epochs = 20,
    verbose = 1)
accuracy score : 0.923722943723


dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.01,
    learn_rate_decays = 0.9,
    epochs = 100,
    verbose = 1)
accuracy score : 0.973246753247

dbn = DBN(
    [trainX.shape[1], 300, 10],
    learn_rates = 0.01,
    learn_rate_decays = 0.9,
    epochs = 1000,
    verbose = 1)
accuracy score : 0.97354978355